In [2]:
import cv2

In [ ]:
def genrate_dataset():
    face_classifier=cv2.CascadeClassifier("haarcascade_frontalface_default.xml")
    def facecrop(img):
        gray=cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
        faces = face_classifier.detectMultiScale(gray,1.3,5)
        #scailing factor=1.3,min height=5
        if faces is():
            return None
        for(x,y,w,h) in faces:
            cropped_face=img[y:y+h,x:x+w]
        return cropped_face
    cap = cv2.VideoCapture(0)
    id=1
    
    img_id=0

    while True:
        ret,frame=cap.read()
        if facecrop(frame) is not None:
            img_id+=1
            face=cv2.resize(facecrop(frame),(200,200))
            face=cv2.cvtColor(face,cv2.COLOR_BGR2GRAY)
            file_name_path="Data/user."+str(id)+"."+str(img_id)+".jpg"
            cv2.imwrite(file_name_path,face)
            cv2.putText(face,str(img_id),(50,50),cv2.FONT_HERSHEY_SCRIPT_COMPLEX,1,(0,255,0),2)
            # (50,50) is origint point from where text is to be written , thickness=2
            cv2.imshow("Croppped face",face)
            if cv2.waitKey(1)==13 or int(img_id)==200:
                break
    cap.release()
    cv2.destroyAllWindows()
    print("Collecting Samples is Completed....")

genrate_dataset()
        
        




<>:7: SyntaxWarning: "is" with 'tuple' literal. Did you mean "=="?
<>:7: SyntaxWarning: "is" with 'tuple' literal. Did you mean "=="?
C:\Users\arane\AppData\Local\Temp\ipykernel_9820\1794511059.py:7: SyntaxWarning: "is" with 'tuple' literal. Did you mean "=="?
  if faces is():


#Train The classifier and save it

In [3]:
import numpy as np
from PIL import Image
import os
import cv2

In [ ]:
def train_classifier(data_dir):
    if not os.path.exists(data_dir):
        print(f"Error: Directory '{data_dir}' does not exist.")
        return

    path = [os.path.join(data_dir, f) for f in os.listdir(data_dir) if f.endswith(('.jpg', '.jpeg', '.png'))]
    faces = []
    ids = []

    for image_path in path:
        try:
            img = Image.open(image_path).convert('L')  # Convert image to grayscale
            imageNp = np.array(img, 'uint8')
            id = int(os.path.split(image_path)[1].split(".")[1])  # Convert id to integer
            faces.append(imageNp)
            ids.append(id)
        except Exception as e:
            print(f"Error processing file {image_path}: {e}")

    if not faces:
        print("Error: No valid face images found!")
        return

    ids = np.array(ids)

    # Train the classifier and save
    clf = cv2.face.LBPHFaceRecognizer_create()
    clf.train(faces, ids)
    clf.write("classifier.xml")

    print("Training completed successfully!")

# Run the function
train_classifier("Data")
        

In [ ]:
def draw_boundary(img, classifier, scalefactor, minNeighbors, color, text, clf):
    gray_image = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    features = classifier.detectMultiScale(gray_image, scalefactor, minNeighbors)

    coords = []

    for (x, y, w, h) in features:
        cv2.rectangle(img, (x, y), (x + w, y + h), color, 2)
        id, pred = clf.predict(gray_image[y:y+h, x:x+w])  # Fixed incorrect indexing

        confidence = int(100 * (1 - pred / 300))
        if confidence > 75:
            if id == 1:
                name = "Atharva"
            elif id == 2:
                name = "Mugdha"
            else:
                name = "Unknown"

            cv2.putText(img, name, (x, y - 5), cv2.FONT_HERSHEY_SIMPLEX, 0.8, color, 1, cv2.LINE_AA)

        coords.append((x, y, w, h))  # Store all detected face coordinates

    return coords  # Return all detected face coordinates

def recognize(img, clf, faceCascade):
    draw_boundary(img, faceCascade, 1.1, 10, (255, 255, 255), "Face", clf)
    return img

# Load Haar cascade for face detection
faceCascade = cv2.CascadeClassifier("haarcascade_frontalface_default.xml")

# Load the trained classifier
clf = cv2.face.LBPHFaceRecognizer_create()
clf.read("classifier.xml")

# Start video capture
video_capture = cv2.VideoCapture(0)

while True:
    ret, img = video_capture.read()
    img = recognize(img, clf, faceCascade)
    cv2.imshow("Face Detection", img)

    if cv2.waitKey(1) == 13:  # Press 'Enter' to exit
        break

video_capture.release()
cv2.destroyAllWindows()

In [1]:
import tkinter as tk
from tkinter import messagebox
import cv2
import os
from PIL import Image
import numpy as np
import mysql.connector

In [2]:
try:
    mydb = mysql.connector.connect(
        host="localhost",
        user="root",
        passwd="Rohit@45",
        database="facerecognise"
    )

    if mydb.is_connected():
        print("Connection Done")
    else:
        print("Bye bye")

except mysql.connector.Error as err:
    print(f"Error: {err}")


Error: Authentication plugin 'caching_sha2_password' is not supported


In [35]:
window=tk.Tk()
window.title("Face Recognition system")
window.config(bg="white")

l1=tk.Label(window,text="Name",font=("Algerian,20"))
l1.grid(column=0,row=0)
t1=tk.Entry(window,width=50,bd=5)
t1.grid(column=1,row=0)

l2=tk.Label(window,text="Age",font=("Algerian,20"))
l2.grid(column=0,row=1)
t2=tk.Entry(window,width=50,bd=5)
t2.grid(column=1,row=1)

l3=tk.Label(window,text="Address",font=("Algerian,20"))
l3.grid(column=0,row=2)
t3=tk.Entry(window,width=50,bd=5)
t3.grid(column=1,row=2)

def train_classifier():
    data_dir="C:/Users/arane/OneDrive/Desktop/HTML TUTORIAL/ML/Data"
    if not os.path.exists(data_dir):
        print(f"Error: Directory '{data_dir}' does not exist.")
        return

    path = [os.path.join(data_dir, f) for f in os.listdir(data_dir) if f.endswith(('.jpg', '.jpeg', '.png'))]
    faces = []
    ids = []

    for image_path in path:
        try:
            img = Image.open(image_path).convert('L')  # Convert image to grayscale
            imageNp = np.array(img, 'uint8')
            id = int(os.path.split(image_path)[1].split(".")[1])  # Convert id to integer
            faces.append(imageNp)
            ids.append(id)
        except Exception as e:
            print(f"Error processing file {image_path}: {e}")

    if not faces:
        print("Error: No valid face images found!")
        return

    ids = np.array(ids)

    # Train the classifier and save
    clf = cv2.face.LBPHFaceRecognizer_create()
    clf.train(faces, ids)
    clf.write("classifier.xml")

    messagebox.showinfo('Result',"Training Done")


        

def genrate_dataset():
    if t1.get() == "" or t2.get() == "" or t3.get() == "":
        messagebox.showwarning("Input Error", "Please fill in all fields before proceeding!")

    else:
         face_classifier=cv2.CascadeClassifier("haarcascade_frontalface_default.xml")
    def facecrop(img):
        gray=cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
        faces = face_classifier.detectMultiScale(gray,1.3,5)
        #scailing factor=1.3,min height=5
        if faces is():
            return None
        for(x,y,w,h) in faces:
            cropped_face=img[y:y+h,x:x+w]
        return cropped_face
    cap = cv2.VideoCapture(0)
    id=1
    
    img_id=0

    while True:
        ret,frame=cap.read()
        if facecrop(frame) is not None:
            img_id+=1
            face=cv2.resize(facecrop(frame),(200,200))
            face=cv2.cvtColor(face,cv2.COLOR_BGR2GRAY)
            file_name_path="Data/user."+str(id)+"."+str(img_id)+".jpg"
            cv2.imwrite(file_name_path,face)
            cv2.putText(face,str(img_id),(50,50),cv2.FONT_HERSHEY_SCRIPT_COMPLEX,1,(0,255,0),2)
            # (50,50) is origint point from where text is to be written , thickness=2
            cv2.imshow("Croppped face",face)
            if cv2.waitKey(1)==13 or int(img_id)==200:
                break
    cap.release()
    cv2.destroyAllWindows()
    messagebox.showinfo('Result','User info is collected')
   

        
        






b1=tk.Button(window,text="Genrate Dataset",font=("Algerian",20),bg='orange',fg='black',height=1,width=10,command=genrate_dataset)
b1.grid(column=0,row=4)


b2=tk.Button(window,text="Train",font=("Algerian",20),bg='Green',fg='black',height=1,width=10,command=train_classifier)
b2.grid(column=1,row=4)

def detect_face():
    def draw_boundary(img, classifier, scalefactor, minNeighbors, color, text, clf):
        gray_image = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        features = classifier.detectMultiScale(gray_image, scalefactor, minNeighbors)

        coords = []

        for (x, y, w, h) in features:
            cv2.rectangle(img, (x, y), (x + w, y + h), color, 2)
            id, pred = clf.predict(gray_image[y:y+h, x:x+w])  # Fixed incorrect indexing
            confidence = int(100 * (1 - pred / 300))
            if confidence > 75:
                if id == 1:
                    name = "Atharva"
                elif id == 2:
                    name = "Mugdha"
                else:
                    name = "Unknown"

                cv2.putText(img, name, (x, y - 5), cv2.FONT_HERSHEY_SIMPLEX, 0.8, color, 1, cv2.LINE_AA)

            coords.append((x, y, w, h))  # Store all detected face coordinates

        return coords  # Return all detected face coordinates

    def recognize(img, clf, faceCascade):
        draw_boundary(img, faceCascade, 1.1, 10, (255, 255, 255), "Face", clf)
        return img

    # Load Haar cascade for face detection
    faceCascade = cv2.CascadeClassifier("haarcascade_frontalface_default.xml")

    # Load the trained classifier
    clf = cv2.face.LBPHFaceRecognizer_create()
    clf.read("classifier.xml")

    # Start video capture
    video_capture = cv2.VideoCapture(0)

    while True:
        ret, img = video_capture.read()
        img = recognize(img, clf, faceCascade)
        cv2.imshow("Face Detection", img)

        if cv2.waitKey(1) == 13:  # Press 'Enter' to exit
            break

    video_capture.release()
    cv2.destroyAllWindows()
    



b3=tk.Button(window,text="Detect The face",font=("Algerian",20),bg='blue',fg='black',height=1,command=detect_face)
b3.grid(column=2,row=4)
window.geometry("800x200")
window.mainloop()



<>:66: SyntaxWarning: "is" with 'tuple' literal. Did you mean "=="?
<>:66: SyntaxWarning: "is" with 'tuple' literal. Did you mean "=="?
C:\Users\arane\AppData\Local\Temp\ipykernel_20840\432631533.py:66: SyntaxWarning: "is" with 'tuple' literal. Did you mean "=="?
  if faces is():
